In [ ]:
import pickle
import mne
from pathlib import Path
import numpy as np
import pandas as pd
import scipy
import os

import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import mne
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test

from plotly import tools
from plotly.graph_objs import Layout, YAxis, Scatter, Annotation, Annotations, Data, Figure, Marker, Font

In [ ]:
eeg_electrodes_32 = ['FP1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT9', 'FC5', 'FC1', 'FC2', 'FC6', 'FT10', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2']
eeg_electrodes_29 = ['F1', 'F2', 'Fz', 'CP3', 'C3', 'FC3', 'FCz', 'Cz', 'CPz', 'FC4', 'C4', 'CP4', 'FC5', 'C5', 'CP5', 'FC1', 'C1', 'CP1', 'FC2', 'C2', 'CP2', 'FC6', 'C6', 'CP6', 'P4', 'P3', 'P1', 'Pz', 'P2']
common_electrodes = set(eeg_electrodes_32).intersection(set(eeg_electrodes_29))

In [ ]:
data_indexes = {'left_real': [], 'right_real': []}
data_stat = {'subject': [], 'left': [], 'right': []} # subject_trial_ids
data_subjects_32 = {'left_real': [], 'right_real': []} # data without trials
data_subjects_29 = {'left_real': [], 'right_real': []} # data without trials
data_subjects = {'left_real': [], 'right_real': []} # data without trials

Set epoch duration

In [ ]:
epoch_onset = -1
epoch_offset = 3
epoch_duration = epoch_offset - epoch_onset

Read 29 electrodes data

In [ ]:
mne_python_raw = mne.io.read_raw_fif(f'{subject_path}/mne_python_raw.fif')
sample_frequency_29 = mne_python_raw.info['sfreq']
electrodes_names_29 = mne_python_raw.info['ch_names']
epoch_onset_index = int(epoch_onset * sample_frequency_29)
epoch_offset_index = int(epoch_offset * sample_frequency_29)

data_29_path = 'E:/YandexDisk/EEG/raw/MI_NN/29_chanels/Real/'
data_codes = {'left_real': 1.0, 'right_real': 2.0}
subject_id = 0
for subject_path in Path(data_29_path).iterdir():
    data_stat['subject'].append(f"S{subject_id}")
    with open(f'{subject_path}/data_full.pkl', "rb") as f:
        data_full = pickle.load(f, encoding="bytes")
    with open(f'{subject_path}/states_full.pkl', "rb") as f:
        states_full = pickle.load(f, encoding="bytes")
    for movement in data_codes:
        movement_ids = np.argwhere(states_full == data_codes[movement])
        diff_values = np.diff(movement_ids[:, 1])
        split_indexes_ids = np.where(diff_values > 1.0)[0]
        split_indexes = movement_ids[:, 1][split_indexes_ids]
        data_stat[movement[:-5]].append(len(split_indexes_ids) + 1)
        for trial_id in range(0, len(split_indexes_ids) + 1):
            if trial_id == 0:
                trial_start = movement_ids[0, 1]
                trial_end = movement_ids[split_indexes_ids[trial_id], 1] + 1
            elif trial_id == len(split_indexes_ids):
                trial_start = movement_ids[split_indexes_ids[trial_id - 1], 1]
                trial_end = movement_ids[-1, 1] + 1
            else:
                trial_start = movement_ids[split_indexes_ids[trial_id - 1] + 1, 1]
                trial_end = movement_ids[split_indexes_ids[trial_id], 1] + 1
            if trial_id == 0 and states_full[:, trial_start - 1] in [0.0, 3.0]:
                if states_full[:, trial_start+epoch_onset_index] in [0.0, 3.0] and states_full[:, trial_start+epoch_offset_index] == data_codes[movement]:
                    data_subjects_29[movement].append(data_full[:, trial_start+epoch_onset_index:trial_start+epoch_offset_index])
                else:
                    continue
            if trial_id > 0 and states_full[:, trial_start - 1] in [0.0, 3.0]:
                if states_full[:, trial_start+epoch_onset_index] in [0.0, 3.0] and states_full[:, trial_start+epoch_offset_index] == data_codes[movement]:
                    data_subjects_29[movement][subject_id] = np.concatenate((data_subjects_29[movement][subject_id], data_full[:, trial_start+epoch_onset_index:trial_start+epoch_offset_index]), axis=1)
                else:
                    continue
        data_indexes[movement].append(f"S{subject_id}_{movement}_29")
    subject_id += 1

num_subjects_29 = subject_id - 1

Read 32 electrodes data

In [ ]:
from pymatreader import read_mat

data_32_path = 'E:/YandexDisk/EEG/raw/'
data_32_file = '1st_Day.mat'

curr_32_data = read_mat(data_32_path + data_32_file)

In [ ]:
electrodes_names_32 = curr_32_data['subs_ica'][0]['right_real']['label']
sample_frequency_32 = curr_32_data['subs_ica'][0]['right_real']['fsample']
epoch_onset_index = int(epoch_onset * sample_frequency_32)
epoch_offset_index = int(epoch_offset * sample_frequency_32)

for subject_id in range(0, len(curr_32_data['subs_ica'])):
    data_stat['subject'].append(f"S{num_subjects_29 + 1 + subject_id}")
    for key in data_codes:
        data_stat[key[:-5]].append(len(curr_32_data['subs_ica'][subject_id][key]['trial']))
        for trial_id in range(0, len(curr_32_data['subs_ica'][subject_id][key]['trial'])):
            if trial_id == 0:
                data_subjects_32[key].append(curr_32_data['subs_ica'][subject_id][key]['trial'][trial_id][:, 5000+epoch_onset_index:5000+epoch_offset_index])
            if trial_id > 0:
                data_subjects_32[key][subject_id] = np.concatenate((data_subjects_32[key][subject_id], curr_32_data['subs_ica'][subject_id][key]['trial'][trial_id][:, 5000+epoch_onset_index:5000+epoch_offset_index]), axis=1)
        data_indexes[key].append(f"S{num_subjects_29 + 1 + subject_id}_{key}_32")

Data for common electrodes

In [ ]:
electrodes_names = list(set(electrodes_names_32).intersection(set(electrodes_names_29)))
electrodes_32_ids = []
electrodes_29_ids = []
for common_electrode in common_electrodes:
    electrodes_32_ids.append(electrodes_names_32.index(common_electrode))
    electrodes_29_ids.append(electrodes_names_29.index(common_electrode))
for movement in data_codes:
    for subject_id in range(0, len(data_subjects_29[movement])):
        if data_indexes[movement][subject_id].endswith('29'):
            data_subjects[movement].append(data_subjects_29[movement][subject_id][electrodes_29_ids, :])
    for subject_id in range(0, len(data_subjects_32[movement])):
        if data_indexes[movement][len(data_subjects_29[movement]) + subject_id].endswith('32'):
            data_subjects[movement].append(data_subjects_32[movement][subject_id][electrodes_32_ids, :])
if sample_frequency_29 == sample_frequency_32:
    sample_frequency = sample_frequency_32
else:
    print(f'Sample frequencies mismatch! {sample_frequency_29} for 29 electrodes, {sample_frequency_32} for 32 electrodes')

Plot electrodes data MNE

In [ ]:
plot_save_path = 'E:/YandexDisk/EEG/experiments/ERDS/signals/'

if not os.path.exists(plot_save_path):
    os.makedirs(plot_save_path)

ch_names = electrodes_names
sfreq = sample_frequency
ch_types = ['eeg'] * len(ch_names)
montage = mne.channels.make_standard_montage('standard_1005')
info = mne.create_info(ch_names, sfreq, ch_types)
info.set_montage(montage)

for movement in data_codes:
    plot_save_path_mne = f'{plot_save_path}mne/'
    if not os.path.exists(plot_save_path_mne):
        os.makedirs(plot_save_path_mne)
    for subject_id in range(0, len(data_indexes[movement])):
        curr_subject_data = data_subjects[movement][subject_id]
        curr_subject = data_indexes[movement][subject_id]
        num_trials = int(curr_subject_data.shape[1] / (epoch_duration * sample_frequency))
        for trial_id in range(0, num_trials):
            curr_trial_start = int(trial_id * epoch_duration * sample_frequency)
            curr_trial_end = curr_trial_start + int(epoch_duration * sample_frequency)
            curr_trial_data = curr_subject_data[:, curr_trial_start:curr_trial_end]
            
            raw_data = mne.io.RawArray(curr_trial_data, info)

            fig = raw_data.plot(show_scrollbars=False)
            fig.get_axes()[0].axvline(1, linewidth=1, color="black", linestyle=":")
            fig.savefig(f"{plot_save_path_mne}/{curr_subject}_T{trial_id}.png", dpi=400)
            fig.savefig(f"{plot_save_path_mne}/{curr_subject}_T{trial_id}.pdf")

Plot electrodes data Plotly

In [ ]:
for movement in data_codes:
    plot_save_path_plotly = f'{plot_save_path}plotly/'
    if not os.path.exists(plot_save_path_plotly):
        os.makedirs(plot_save_path_plotly)
    for subject_id in range(0, len(data_indexes[movement])):
        curr_subject_data = data_subjects[movement][subject_id]
        curr_subject = data_indexes[movement][subject_id]
        num_trials = int(curr_subject_data.shape[1] / (epoch_duration * sample_frequency))
        for trial_id in range(0, num_trials):
            curr_trial_start = int(trial_id * epoch_duration * sample_frequency)
            curr_trial_end = curr_trial_start + int(epoch_duration * sample_frequency)
            curr_trial_data = curr_subject_data[:, curr_trial_start:curr_trial_end]
            
            raw_data = mne.io.RawArray(curr_trial_data, info)
            
            picks = mne.pick_types(info, eeg=True, exclude=[])
            start, stop = raw_data.time_as_index([0, epoch_duration])
            
            n_channels = len(common_electrodes)
            data, times = raw_data[picks[:n_channels], start:stop]
            ch_names = [raw_data.info['ch_names'][p] for p in picks[:n_channels]]
            
            step = 1.0 / len(common_electrodes)
            kwargs = dict(domain=[1 - step, 1], showticklabels=False, zeroline=False, showgrid=False)
            
            # create objects for layout and traces
            layout = Layout(yaxis=YAxis(kwargs), showlegend=False)
            traces = [Scatter(x=times, y=data.T[:, 0])]
            
            # loop over the channels
            for ii in range(1, n_channels):
                    kwargs.update(domain=[1 - (ii + 1) * step, 1 - ii * step])
                    layout.update({'yaxis%d' % (ii + 1): YAxis(kwargs), 'showlegend': False})
                    traces.append(Scatter(x=times, y=data.T[:, ii], yaxis='y%d' % (ii + 1)))
            
            # add channel names using Annotations
            annotations = Annotations([Annotation(x=-0.06, y=0, xref='paper', yref='y%d' % (ii + 1),
                                                  text=ch_name, font=Font(size=9), showarrow=False)
                                      for ii, ch_name in enumerate(ch_names)])
            layout.update(annotations=annotations)
            
            # set the size of the figure and plot it
            layout.update(autosize=False, width=1000, height=600)
            fig = Figure(data=Data(traces), layout=layout)
            fig.write_image(f"{plot_save_path_plotly}/{curr_subject}_T{trial_id}.png")
            fig.write_image(f"{plot_save_path_plotly}/{curr_subject}_T{trial_id}.pdf")

Plot PSD mne

In [ ]:
plot_save_path = 'E:/YandexDisk/EEG/experiments/ERDS/psd/'

if not os.path.exists(plot_save_path):
    os.makedirs(plot_save_path)

ch_names = electrodes_names
sfreq = sample_frequency
ch_types = ['eeg'] * len(ch_names)
montage = mne.channels.make_standard_montage('standard_1005')
info = mne.create_info(ch_names, sfreq, ch_types)
info.set_montage(montage)

for movement in data_codes:
    plot_save_path_mne = f'{plot_save_path}mne/'
    if not os.path.exists(plot_save_path_mne):
        os.makedirs(plot_save_path_mne)
    for subject_id in range(0, len(data_indexes[movement])):
        curr_subject_data = data_subjects[movement][subject_id]
        curr_subject = data_indexes[movement][subject_id]
        
        raw_data = mne.io.RawArray(curr_subject_data, info)
        spectrum = raw_data.compute_psd()
        
        plot_save_path_averaged = f'{plot_save_path_mne}averaged/'
        if not os.path.exists(plot_save_path_averaged):
            os.makedirs(plot_save_path_averaged)
        
        fig = spectrum.plot(average=True, picks="data")
        fig.savefig(f"{plot_save_path_averaged}/{curr_subject}.png", dpi=400)
        fig.savefig(f"{plot_save_path_averaged}/{curr_subject}.pdf")
        plt.close()
        
        plot_save_path_all_electrodes = f'{plot_save_path_mne}all_electrodes/'
        if not os.path.exists(plot_save_path_all_electrodes):
            os.makedirs(plot_save_path_all_electrodes)
        
        fig = spectrum.plot(average=False, spatial_colors=True, picks="data")
        fig.savefig(f"{plot_save_path_all_electrodes}/{curr_subject}.png", dpi=400)
        fig.savefig(f"{plot_save_path_all_electrodes}/{curr_subject}.pdf")
        plt.close()
        
        num_trials = int(curr_subject_data.shape[1] / (epoch_duration * sample_frequency))
        for trial_id in range(0, num_trials):
            curr_trial_start = int(trial_id * epoch_duration * sample_frequency)
            curr_trial_end = curr_trial_start + int(epoch_duration * sample_frequency)
            curr_trial_data = curr_subject_data[:, curr_trial_start:curr_trial_end]
            
            raw_data = mne.io.RawArray(curr_trial_data, info)
            spectrum = raw_data.compute_psd()
            
            if not os.path.exists(f'{plot_save_path_averaged}trials/'):
                os.makedirs(f'{plot_save_path_averaged}trials/')
            
            fig = spectrum.plot(average=True, picks="data")
            fig.savefig(f"{plot_save_path_averaged}trials/{curr_subject}_T{trial_id}.png", dpi=400)
            fig.savefig(f"{plot_save_path_averaged}trials/{curr_subject}_T{trial_id}.pdf")
            plt.close()
            
            if not os.path.exists(f'{plot_save_path_all_electrodes}trials/'):
                os.makedirs(f'{plot_save_path_all_electrodes}trials/')
            
            fig = spectrum.plot(average=False, spatial_colors=True, picks="data")
            fig.savefig(f"{plot_save_path_all_electrodes}trials/{curr_subject}_T{trial_id}.png", dpi=400)
            fig.savefig(f"{plot_save_path_all_electrodes}trials/{curr_subject}_T{trial_id}.pdf")
            plt.close()

Plot PSD scipy

In [ ]:
plot_save_path = 'E:/YandexDisk/EEG/experiments/ERDS/psd/'

if not os.path.exists(plot_save_path):
    os.makedirs(plot_save_path)

for movement in data_codes:
    plot_save_path_scipy = f'{plot_save_path}scipy/'
    if not os.path.exists(plot_save_path_scipy):
        os.makedirs(plot_save_path_scipy)
    for subject_id in range(0, len(data_indexes[movement])):
        curr_subject_data = data_subjects[movement][subject_id]
        curr_subject = data_indexes[movement][subject_id]
        
        plot_save_path_averaged = f'{plot_save_path_scipy}averaged/'
        if not os.path.exists(plot_save_path_averaged):
            os.makedirs(plot_save_path_averaged)
        
        freqs, psds = scipy.signal.welch(curr_subject_data, sample_frequency, nperseg=4*1024)
        fig = plt.figure()
        plt.semilogy(freqs, np.average(psds, axis=0), linewidth=0.5)
        plt.xlim([0, 500])
        plt.xlabel('frequency [Hz]')
        plt.ylabel('PSD [V**2/Hz]')
        
        fig.savefig(f"{plot_save_path_averaged}/{curr_subject}.png", dpi=400)
        fig.savefig(f"{plot_save_path_averaged}/{curr_subject}.pdf")
        plt.close()

        num_trials = int(curr_subject_data.shape[1] / (epoch_duration * sample_frequency))
        for trial_id in range(0, num_trials):
            curr_trial_start = int(trial_id * epoch_duration * sample_frequency)
            curr_trial_end = curr_trial_start + int(epoch_duration * sample_frequency)
            curr_trial_data = curr_subject_data[:, curr_trial_start:curr_trial_end]
            
            freqs, psds = scipy.signal.welch(curr_trial_data, sample_frequency, nperseg=4*1024)
            
            fig = plt.figure()
            plt.semilogy(freqs, np.average(psds, axis=0), linewidth=0.5)
            plt.xlim([0, 500])
            plt.xlabel('frequency [Hz]')
            plt.ylabel('PSD [V**2/Hz]')
            
            if not os.path.exists(f'{plot_save_path_averaged}trials/'):
                os.makedirs(f'{plot_save_path_averaged}trials/')
            
            fig.savefig(f"{plot_save_path_averaged}trials/{curr_subject}_T{trial_id}.png", dpi=400)
            fig.savefig(f"{plot_save_path_averaged}trials/{curr_subject}_T{trial_id}.pdf")
            plt.close()

Compute and plot ERDS

In [ ]:
plot_save_path = 'E:/YandexDisk/EEG/experiments/ERDS/erds_maps/'

if not os.path.exists(plot_save_path):
    os.makedirs(plot_save_path)

ch_names = electrodes_names
sfreq = sample_frequency
ch_types = ['eeg'] * len(ch_names)
info = mne.create_info(ch_names, sfreq, ch_types)

for movement in data_codes:

    for subject_id in range(0, len(data_indexes[movement])):
    
        raw_data = mne.io.RawArray(data_subjects[movement][subject_id], info)
        curr_index = data_indexes[movement][subject_id]
        
        events = mne.make_fixed_length_events(raw_data, id=1, start=1.0, stop=None, duration=4.0, first_samp=True, overlap=0.0)
        
        if movement == 'left_real':
            event_ids = dict(left_real=1)
        else:
            event_ids = dict(right_real=1) 
        t_min = -1
        t_max = 3
        epochs = mne.Epochs(raw_data, events, event_id=event_ids, tmin=t_min, tmax=t_max, baseline=None, preload=True)
        
        freqs = np.arange(2, 36)  # frequencies from 2-35Hz
        vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
        baseline = (-1, 0)  # baseline interval (in s)
        cnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # min, center & max ERDS
        
        kwargs = dict(
            n_permutations=100, step_down_p=0.05, seed=1, buffer_size=None, out_type="mask"
        )  # for cluster test
        
        tfr = tfr_multitaper(
            epochs,
            freqs=freqs,
            n_cycles=freqs,
            use_fft=True,
            return_itc=False,
            average=False,
            decim=2,
        )
        tfr.crop(t_min, t_max).apply_baseline(baseline, mode="percent")
        
        for event in event_ids:
            # select desired epochs for visualization
            tfr_ev = tfr[event]
            fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(9, 16))
            plt.rcParams.update({'font.size': 10})
            for ch_id in range(0, len(common_electrodes)):  # for each channel
                # positive clusters
                _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch_id], tail=1, **kwargs)
                # negative clusters
                _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch_id], tail=-1, **kwargs)
        
                # note that we keep clusters with p <= 0.05 from the combined clusters
                # of two independent tests; in this example, we do not correct for
                # these two comparisons
                c = np.stack(c1 + c2, axis=2)  # combined clusters
                p = np.concatenate((p1, p2))  # combined p-values
                mask = c[..., p <= 0.05].any(axis=-1)
                
                row_id = ch_id // 3
                column_id = ch_id % 3
        
                # plot TFR (ERDS map with masking)
                tfr_ev.average().plot(
                    [ch_id],
                    cmap="RdBu",
                    cnorm=cnorm,
                    axes=axes[row_id, column_id],
                    colorbar=True,
                    show=False,
                    mask=mask,
                    mask_style="mask",
                )
                axes[row_id, column_id].set_title(epochs.ch_names[ch_id], loc='left')
                axes[row_id, column_id].axvline(0, linewidth=1, color="black", linestyle=":")  # event
            
            plt.rcParams.update({'font.size': 10})
            fig.suptitle(f"ERDS ({event}) \n")
            plt.tight_layout()
            
            fig.savefig(f"{plot_save_path}/{curr_index}.png", dpi=400)
            fig.savefig(f"{plot_save_path}/{curr_index}.pdf")